In [1]:
"""
Proyecto: Fenotipado y Predicción de Severidad en Síndrome Crónico
Autor: Camiau
Descripción:
- Simulación de dataset de alta dimensionalidad (15 features + Outcome binario de Severidad)
- Escalado + PCA para explorar estructura latente
- Fenotipado no supervisado con KMeans (3 clusters)
- Modelo supervisado con Random Forest usando Fenotipo como predictor adicional
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, RocCurveDisplay


def simular_dataset(n_muestras: int = 1000, random_state: int = 42) -> pd.DataFrame:
    """
    Simula un dataset con 15 biomarcadores/síntomas (X1...X15)
    y un outcome binario de severidad (Severidad: 0=Baja, 1=Alta).
    """
    X, y = make_classification(
        n_samples=n_muestras,
        n_features=15,
        n_informative=8,
        n_redundant=3,
        n_repeated=0,
        n_classes=2,
        n_clusters_per_class=2,
        class_sep=1.5,
        flip_y=0.03,
        random_state=random_state,
    )

    columnas = [f"Feature_{i}" for i in range(1, 16)]
    df = pd.DataFrame(X, columns=columnas)
    df["Severidad"] = y  # 1 = Alta, 0 = Baja
    return df


def escalado_y_pca(df: pd.DataFrame):
    """
    - Escala las 15 features con StandardScaler.
    - Aplica PCA para reducir a 2 componentes principales.
    - Devuelve:
        X_scaled: matriz (n x 15) escalada
        X_pca: matriz (n x 2) con PC1 y PC2
        pca: objeto PCA para inspeccionar varianza explicada
        scaler: StandardScaler ajustado (por si se quiere reutilizar)
    """
    features = [col for col in df.columns if col.startswith("Feature_")]

    X = df[features].values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)

    var_exp_total = pca.explained_variance_ratio_.sum()
    print("=== PCA ===")
    print(f"Varianza explicada por PC1: {pca.explained_variance_ratio_[0]:.3f}")
    print(f"Varianza explicada por PC2: {pca.explained_variance_ratio_[1]:.3f}")
    print(f"Varianza explicada total (PC1+PC2): {var_exp_total:.3f}\n")

    return X_scaled, X_pca, pca, scaler


def plot_pca_severidad(X_pca, severidad, filename: str = "fig_pca_severidad.png"):
    """
    Genera un gráfico de dispersión PC1 vs PC2 coloreado por severidad real.
    Guarda la figura como PNG para adjuntar al repo.
    """
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(
        X_pca[:, 0],
        X_pca[:, 1],
        c=severidad,
        cmap="coolwarm",
        alpha=0.7,
        edgecolor="k",
        s=50,
    )
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.title("Proyección PCA (PC1 vs PC2) coloreada por Severidad real")
    cbar = plt.colorbar(scatter)
    cbar.set_label("Severidad (0=Baja, 1=Alta)")
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.close()
    print(f"Gráfico PCA vs Severidad guardado en: {filename}\n")


def fenotipado_kmeans(df: pd.DataFrame, X_scaled, n_clusters: int = 3, random_state: int = 42):
    """
    Aplica KMeans sobre los datos escalados (X_scaled) para descubrir 3 fenotipos.
    Devuelve el DataFrame con una nueva columna 'Fenotipo'.
    """
    print("=== K-Means ===")
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=random_state)
    etiquetas = kmeans.fit_predict(X_scaled)
    df["Fenotipo"] = etiquetas
    print(f"Fenotipos asignados (0 a {n_clusters-1})\n")
    return df, kmeans


def plot_pca_fenotipos(X_pca, fenotipos, filename: str = "fig_pca_fenotipos.png"):
    """
    Gráfico PC1 vs PC2 coloreado por fenotipo KMeans.
    No es obligatorio, pero ayuda a visualizar qué tan distintos son los clusters.
    """
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(
        X_pca[:, 0],
        X_pca[:, 1],
        c=fenotipos,
        cmap="viridis",
        alpha=0.7,
        edgecolor="k",
        s=50,
    )
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.title("Proyección PCA (PC1 vs PC2) coloreada por Fenotipo (K-Means)")
    cbar = plt.colorbar(scatter)
    cbar.set_label("Fenotipo KMeans")
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.close()
    print(f"Gráfico PCA vs Fenotipos guardado en: {filename}\n")


def modelo_random_forest(df: pd.DataFrame, random_state: int = 42):
    """
    - Divide los datos en Train/Test (70/30), incluyendo 'Fenotipo' como predictor.
    - Ajusta un RandomForestClassifier para predecir Severidad.
    - Reporta AUC final en test.
    """
    from sklearn.metrics import RocCurveDisplay

    features = [col for col in df.columns if col.startswith("Feature_")]
    features_con_fenotipo = features + ["Fenotipo"]

    X = df[features_con_fenotipo].values
    y = df["Severidad"].values

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.3,
        random_state=random_state,
        stratify=y,
    )

    print("=== Random Forest (con Fenotipo) ===")
    rf = RandomForestClassifier(
        n_estimators=200,
        max_depth=None,
        min_samples_leaf=5,
        random_state=random_state,
        n_jobs=-1,
    )

    rf.fit(X_train, y_train)
    y_proba_test = rf.predict_proba(X_test)[:, 1]

    auc_test = roc_auc_score(y_test, y_proba_test)
    print(f"AUC Final en Test: {auc_test:.3f}\n")

    # Curva ROC 
    plt.figure(figsize=(6, 6))
    RocCurveDisplay.from_estimator(rf, X_test, y_test)
    plt.title("Curva ROC - Random Forest (Test)")
    plt.tight_layout()
    plt.savefig("fig_roc_random_forest.png", dpi=300)
    plt.close()
    print("Curva ROC guardada en: fig_roc_random_forest.png\n")

    return rf, auc_test


def main():
    # 1. Simular dataset
    df = simular_dataset()

    # 2. Escalado + PCA + visualización
    X_scaled, X_pca, pca, scaler = escalado_y_pca(df)
    plot_pca_severidad(X_pca, df["Severidad"].values)

    # 3. Fenotipado con KMeans (sobre datos escalados)
    df, kmeans = fenotipado_kmeans(df, X_scaled, n_clusters=3)
    plot_pca_fenotipos(X_pca, df["Fenotipo"].values)

    # 4. Modelo supervisado (Random Forest) usando Fenotipo como predictor extra
    rf_model, auc_test = modelo_random_forest(df)

    print("Pipeline completo ejecutado.\n")


if __name__ == "__main__":
    main()


=== PCA ===
Varianza explicada por PC1: 0.209
Varianza explicada por PC2: 0.145
Varianza explicada total (PC1+PC2): 0.354

Gráfico PCA vs Severidad guardado en: fig_pca_severidad.png

=== K-Means ===


C:\Users\draca\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


Fenotipos asignados (0 a 2)

Gráfico PCA vs Fenotipos guardado en: fig_pca_fenotipos.png

=== Random Forest (con Fenotipo) ===
AUC Final en Test: 0.958

Curva ROC guardada en: fig_roc_random_forest.png

Pipeline completo ejecutado.

Recuerda revisar las figuras PNG generadas para la discusión en el README.


<Figure size 600x600 with 0 Axes>